In [1]:
import math
from collections import Counter

import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Download stopwords
stopwords.words('english')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/vscode/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
def preprocess(text, stop_words, stemmer):
    """Text preprocessing pipeline"""
    tokens = nltk.word_tokenize(text.lower())
    tokens = [t for t in tokens if t not in stop_words and t.isalnum()]
    if stemmer:
        return [stemmer.stem(t) for t in tokens]
    return tokens

def qaf(query, term, stop_words, stemmer):
    """Query Absolute Frequency"""
    return preprocess(query, stop_words, stemmer).count(term)
    
def qrf(query, term, stop_words, stemmer):
    """Query Relative Frequency"""
    tokens = preprocess(query, stop_words, stemmer)
    return qaf(query, term, stop_words, stemmer) / len(tokens) if tokens else 0
    
def daf(document, term, stop_words, stemmer):
    """Document Absolute Frequency"""
    return preprocess(document, stop_words, stemmer).count(term)
    
def drf(document, term, stop_words, stemmer):
    """Document Relative Frequency"""
    tokens = preprocess(document, stop_words, stemmer)
    return daf(document, term, stop_words, stemmer) / len(tokens) if tokens else 0
    
def idf(term, tfidf_matrix, vectorizer, total_docs):
    """Inverse Document Frequency (with smoothing)"""
    if term in vectorizer.vocabulary_:
        doc_count = (tfidf_matrix[:, vectorizer.vocabulary_[term]] > 0).sum()
        return math.log((total_docs + 1) / (doc_count + 1)) + 1
    return 0
    
def rfad(term, term_collection_counts, collection_length):
    """Relative Frequency in All Documents"""
    return term_collection_counts.get(term, 0) / collection_length if collection_length else 0

In [4]:
file_name ="ioinc_dataset-v3.xlsx" # path to file + file name
sheet = "Sheet 1" # sheet name or sheet number or list of sheet numbers and names

df = pd.read_excel(file_name, sheet_name=sheet)
print(df.head())

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
stemmer = PorterStemmer() if stem else None
stop_words = set(stopwords.words(language))
documents = documents
total_docs = len(documents)
        
terms = set(preprocess(query, stop_words, stemmer))